In [1]:
import os
import subprocess as sp
from iterextras import par_for
from multiprocessing import cpu_count
from tqdm.auto import tqdm
from pathlib import Path

In [2]:
CRATES = [
    ('Rocket', 'rocket', ''),
    ('image', None, ''),
    ('rayon', 'rayon', None),
    ('rg3d', 'rg3d', None),    
    ('nalgebra', 'nalgebra', '--features rand,arbitrary,sparse,debug,io,libm'), # avoid no_unsound_assume_init feature!
    ('rustls', 'rustls', None),
    ('sccache', None, ''),
    ('hyper', 'hyper', '--features full'),    
    ('rav1e', 'rav1e', ''),
    ('RustPython', 'rustpython-vm', '--features compiler'),
]

In [3]:
NUM_THREADS = 1
NUM_WORKERS = len(CRATES)
CLEAN = False
TOOLCHAIN = 'nightly-2022-02-17'

Path("../data/logs").mkdir(parents=True, exist_ok=True)

def analyze_crate(crate):
    (name, sub_crate, feature_flags) = crate
    out_path = os.path.abspath(f'../data/{name.replace("/", "_")}.json')
    working_dir = f'../data/repos/{name}'
    log_path = os.path.abspath(f'../data/logs/{name}.log')
    
    if os.path.exists(out_path) and os.stat(out_path).st_size > 0:
        return
    
    if CLEAN:
        sp.check_call('cargo clean && rm -rf target', cwd=working_dir, shell=True)
    
    rustc = f'$(rustup which --toolchain {TOOLCHAIN} rustc)'
    ld_lib_path = f'LD_LIBRARY_PATH=$({rustc} --print target-libdir):$LD_LIBRARY_PATH'
    sysroot = f'SYSROOT=$({rustc} --print sysroot)'
    feature_flags = feature_flags if feature_flags is not None else "--all-features"
    pkg_flag = f'-p {sub_crate}' if sub_crate is not None else ''
    cmd = f'{ld_lib_path} {sysroot} OUTPUT_PATH={out_path} RUST_LOG="flowistry_eval=info" \
THREADS={NUM_THREADS} TOOLCHAIN={TOOLCHAIN} RUST_BACKTRACE=1 cargo flowistry-eval --lib {pkg_flag} {feature_flags}'
    log = open(log_path, 'w')
    try:
        print(cmd)
        sp.check_call(cmd, shell=True, cwd=working_dir, stdout=log, stderr=log)
    except sp.CalledProcessError as e:
        print(f'{name} failed')

In [ ]:
par_for(analyze_crate, CRATES, workers=NUM_WORKERS)

LD_LIBRARY_PATH=$($(rustup which --toolchain nightly-2022-02-17 rustc) --print target-libdir):$LD_LIBRARY_PATH SYSROOT=$($(rustup which --toolchain nightly-2022-02-17 rustc) --print sysroot) OUTPUT_PATH=/raid/wcrichto/flowistry-oopsla-experiments/data/Rocket.json RUST_LOG="flowistry_eval=info" THREADS=1 TOOLCHAIN=nightly-2022-02-17 RUST_BACKTRACE=1 cargo flowistry-eval --lib -p rocket 
LD_LIBRARY_PATH=$($(rustup which --toolchain nightly-2022-02-17 rustc) --print target-libdir):$LD_LIBRARY_PATH SYSROOT=$($(rustup which --toolchain nightly-2022-02-17 rustc) --print sysroot) OUTPUT_PATH=/raid/wcrichto/flowistry-oopsla-experiments/data/image.json RUST_LOG="flowistry_eval=info" THREADS=1 TOOLCHAIN=nightly-2022-02-17 RUST_BACKTRACE=1 cargo flowistry-eval --lib  
LD_LIBRARY_PATH=$($(rustup which --toolchain nightly-2022-02-17 rustc) --print target-libdir):$LD_LIBRARY_PATH SYSROOT=$($(rustup which --toolchain nightly-2022-02-17 rustc) --print sysroot) OUTPUT_PATH=/raid/wcrichto/flowistry-oops

  0%|          | 0/10 [00:00<?, ?it/s]